# 长短期记忆网络策略

长短期记忆网络（[LSTM](https://en.wikipedia.org/wiki/Long_short-term_memory)）是一种特殊的循环神经网络，可以用于处理时间序列数据。在本教程中，我们将使用 LSTM 网络来预测股票价格，并使用预测结果来构建一个交易策略。

In [ ]:
import sys
import yaml
import os
sys.path.append("..")

In [ ]:
# import
from models.lstm import LSTMModel
from strategies.lstm import LSTMStrategy
from data.read_data import DataReader

In [ ]:
# initialize
with open("../../config/stock_inference.yaml") as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)

下面开始训练模型。

In [ ]:
from train import Trainer
with open("../../config/train.yaml") as f:
    train_cfg = yaml.load(f, Loader=yaml.FullLoader)

trainer = Trainer(cfg)
trainer.train()

下面我们进行股票价格预测，由于我这里定义的模型较为简单，所以发生了一定的过拟合现象。用户可以定义一系列超参数，使模型更加精准（但同时也需要更长时间训练）。

In [ ]:
# inference
from backtest import BackTest
bt = BackTest(cfg)

%matplotlib inline
bt.inference()
bt.plot(mode="inference")

最后我们根据预测的股价进行回测交易。

In [ ]:
strategy = LSTMStrategy(cfg)
reader = DataReader(cfg)
data = reader.get_source_data()
trade_duration = 365 * 1
net_worth = []
time_series = []
for i in range(trade_duration):
    strategy.time = data.index[i]
    data_slice = data.iloc[i]
    stock_price = {}
    for j in range(len(reader.name)):
        stock_price[reader.name[j]] = data_slice[j]

    strategy.trade(stock_price)
    print(f"{strategy.time} net worth: {strategy.get_net(stock_price)} asset: {strategy.asset} stock: {strategy.stock}")
    net_worth.append(strategy.get_net(stock_price))
    time_series.append(strategy.time)